# import

In [73]:
import os
import random
import glob
import re

import pandas as pd
import numpy as np

from sklearn.preprocessing import RobustScaler

import torch
import torch.nn as nn
from tqdm import tqdm

In [74]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import font_manager

font_path="c:/Windows/Fonts/malgun.ttf"
font_name = font_manager.FontProperties(fname=font_path).get_name()
matplotlib.rc('font',family=font_name)

plt.rcParams['font.size'] = 13  # 기본 폰트 크기
plt.rcParams['axes.labelsize'] = 13  # x,y축 label 폰트 크기
plt.rcParams['xtick.labelsize'] = 13  # x축 눈금 폰트 크기
plt.rcParams['ytick.labelsize'] = 13  # y축 눈금 폰트 크기
plt.rcParams['legend.fontsize'] = 13  # 범례 폰트 크기
plt.rcParams['figure.titlesize'] = 15  # figure title 폰트 크기

import colorsys

def generate_colors(n):
    return [colorsys.hsv_to_rgb(i/n, 0.7, 0.9) for i in range(n)]

In [75]:
# GPU 이름 출력
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
else:
    print("GPU is not available, using CPU.")

GPU: NVIDIA GeForce RTX 4080 Laptop GPU


# constant

In [76]:
LOOKBACK = 28
PREDICT = 7
BATCH_SIZE = 32
EPOCHS = 100

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# def

In [77]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(42)

In [78]:
import pandas as pd

# 성수기 구분 데이터 생성 함수
def generate_peak_season_data(df):
    # 업장+메뉴 단위로 월별 평균 매출 계산
    monthly_avg = (
        df.groupby(["영업장명", "메뉴명", "월"])["매출수량"]
        .mean()
        .reset_index()
    )

    # 각 조합별 평균 매출
    base_avg = (
        monthly_avg.groupby(["영업장명", "메뉴명"])["매출수량"]
        .mean()
        .reset_index()
        .rename(columns={"매출수량": "기준매출"})
    )

    # 병합 후 성수기/비수기 구분 (임계치: 평균 대비 1.3배 이상 = 성수기, 0.7배 이하 = 비수기)
    monthly_labeled = pd.merge(monthly_avg, base_avg, on=["영업장명", "메뉴명"])
    monthly_labeled["성수기여부"] = monthly_labeled.apply(
        lambda row: "성수기" if row["매출수량"] >= row["기준매출"] * 1.3
        else "비수기" if row["매출수량"] <= row["기준매출"] * 0.7
        else "구분 불가",
        axis=1
    )
    # 결과: 업장+메뉴+월별 성수기 정보
    return monthly_labeled[["영업장명", "메뉴명", "월", "성수기여부"]].copy()


def preprocess_sales_data_first(df):
    # 날짜 변환
    df["영업일자"] = pd.to_datetime(df["영업일자"])

    # 영업장명과 메뉴명 분리
    df["영업장명"] = df["영업장명_메뉴명"].apply(lambda x: x.split("_")[0])
    df["메뉴명"] = df["영업장명_메뉴명"].apply(lambda x: x.split("_")[1])

    # 요일 및 주말 여부
    df["월"] = df["영업일자"].dt.month
    df["요일"] = df["영업일자"].dt.weekday
    df["주말여부"] = df["요일"].isin([5, 6]).astype(int)

    return df


def preprocess_sales_data_second(df, peak_season_map=None):
    # 공휴일 하드코딩 (2023~2025)
    holidays_2023_2025 = pd.to_datetime([
        # 2023년
        "2023-01-01", "2023-01-21", "2023-01-22", "2023-01-23", "2023-03-01", "2023-05-05",
        "2023-05-27", "2023-06-06", "2023-08-15", "2023-09-28", "2023-09-29", "2023-09-30",
        "2023-10-03", "2023-10-09", "2023-12-25",
        # 2024년
        "2024-01-01", "2024-02-09", "2024-02-10", "2024-02-11", "2024-03-01", "2024-05-05",
        "2024-05-06", "2024-06-06", "2024-08-15", "2024-09-16", "2024-09-17", "2024-09-18",
        "2024-10-03", "2024-10-09", "2024-12-25",
        # 2025년
        "2025-01-01", "2025-01-28", "2025-01-29", "2025-01-30", "2025-03-01", "2025-05-05",
        "2025-06-06", "2025-08-15", "2025-10-03", "2025-10-06", "2025-10-07", "2025-10-08",
        "2025-10-09", "2025-12-25"
    ])
    df["공휴일여부"] = df["영업일자"].isin(holidays_2023_2025).astype(int)

    # 성수기 여부 판별
    # 월 컬럼 생성 후 성수기 맵과 병합
    df["월"] = df["영업일자"].dt.month
    df = df.merge(
        peak_season_map,
        how="left",
        on=["영업장명", "메뉴명", "월"]
    )

    # 성수기여부 결측값은 "구분 불가" 처리
    df["성수기여부"] = df["성수기여부"].fillna("구분 불가")

    # time_idx 추가 (시계열 인덱스)
    df = df.sort_values(["영업장명", "메뉴명", "영업일자"])
    df["time_idx"] = df.groupby(["영업장명", "메뉴명"]).cumcount()

    # group_id 추가
    df["group_id"] = df["영업장명"] + "_" + df["메뉴명"]

    return df

# load

In [79]:
import pandas as pd

# 데이터 불러오기
train = pd.read_csv("data/train.csv")
train = preprocess_sales_data_first(train)
# 성수기 구분 데이터 생성
peak_season_map = generate_peak_season_data(train)
train = preprocess_sales_data_second(train, peak_season_map)

list_name = train["영업장명"].explode().unique()

train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102676 entries, 0 to 102675
Data columns (total 12 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   영업일자      102676 non-null  datetime64[ns]
 1   영업장명_메뉴명  102676 non-null  object        
 2   매출수량      102676 non-null  int64         
 3   영업장명      102676 non-null  object        
 4   메뉴명       102676 non-null  object        
 5   월         102676 non-null  int32         
 6   요일        102676 non-null  int32         
 7   주말여부      102676 non-null  int64         
 8   공휴일여부     102676 non-null  int64         
 9   성수기여부     102676 non-null  object        
 10  time_idx  102676 non-null  int64         
 11  group_id  102676 non-null  object        
dtypes: datetime64[ns](1), int32(2), int64(4), object(5)
memory usage: 8.6+ MB


In [80]:
train.head().T

,0,1,2,3,4
영업일자,2023-01-01 00:00:00,2023-01-02 00:00:00,2023-01-03 00:00:00,2023-01-04 00:00:00,2023-01-05 00:00:00
영업장명_메뉴명,느티나무 셀프BBQ_1인 수저세트,느티나무 셀프BBQ_1인 수저세트,느티나무 셀프BBQ_1인 수저세트,느티나무 셀프BBQ_1인 수저세트,느티나무 셀프BBQ_1인 수저세트
매출수량,0,0,0,0,0
영업장명,느티나무 셀프BBQ,느티나무 셀프BBQ,느티나무 셀프BBQ,느티나무 셀프BBQ,느티나무 셀프BBQ
메뉴명,1인 수저세트,1인 수저세트,1인 수저세트,1인 수저세트,1인 수저세트
월,1,1,1,1,1
요일,6,0,1,2,3
주말여부,1,0,0,0,0
공휴일여부,1,0,0,0,0
성수기여부,비수기,비수기,비수기,비수기,비수기


# 데이터 준비 및 모델 학습

# 모델 평가 및 예측